In [1]:
import pandas as pd
import numpy as np
import math
from funcoes import *
pd.options.mode.chained_assignment = None

#dados foram baixados de https://www.cryptodatadownload.com/ 
#indicadores tecnicos de https://www.akademiabaru.com/doc/ARBMSV14_N1_P35_41.pdf
#indicadores são:  MACD, RSI, SO, MA, EMA, ROC e VT.


acuracias = []

df = pd.read_csv("Binance_BTCUSDT_minute.csv",skiprows=1,nrows=2000) 

calculate_indicators(df) #calcula os indicadores no df e retorna uma lista para ser usado no fit do modelo

df = df.dropna(axis=0,how="any")
x,y = get_x_y(df)


In [2]:
interval = 1000 #decidir qual o melhor interval
print(len(x))
print(len(y))
acertos = 0
lista_resultados = []
for i in range(interval,len(x)):
    #row_70 = math.floor(0.7*interval)+(i*interval)

    x_train,y_train = x[i-interval:i],y[i-interval:i]
    x_test = x[i]
    

    model = LogisticRegression(max_iter=1000)
    model.fit(x_train,y_train)
    coeficientes = list(model.coef_[0])


    #lista_resultados = []
    #for i in x_test:
    lista_resultados.append(calcula_p(x_test,coeficientes))
    
y = y[interval:] #retirando os primeiros 30 elementos do y para poder comparar a lista de resultados usando o mesmo indice.
    
for i in range(0,len(lista_resultados)):
    if lista_resultados[i] >= 0.5 and y[i] == 1:
        acertos += 1
    if lista_resultados[i] < 0.5 and y[i] == 0:
        acertos += 1

print(acertos/len(y)*100)


1975
1975
70.25641025641025
